In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import dask

import glob
from cdo import *
import os
import sys
from datetime import datetime

import cftime

sys.path.append('/glade/u/home/currierw/cmip_ingest/scripts')
from download import get_dataset
sys.path.append('/glade/u/home/currierw/cmip_work')
import cmipFunctions

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 5]
cdo = Cdo()

# Import state vector file using geopandas
states_url = 'http://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_040_00_5m.json'
states_gdf = gpd.read_file(states_url)

In [ ]:
# Parameters
# Slice/Subset data down further based on latitutde (-90 -- +90), longitude (0 -- 360)

#####################################################
############## ADDITIONAL INFORMATION ############## 
#####################################################

#                   [{"var_name":"va", "domain":"atmos", "interval":"6hr"},
#                   {"var_name":"ua", "domain":"atmos", "interval":"6hr"},
#                   {"var_name":"ta", "domain":"atmos", "interval":"6hr"},
#                   {"var_name":"ps", "domain":"atmos", "interval":"6hr"},
#                   {"var_name":"hus", "domain":"atmos", "interval":"6hr"},
#                   {"var_name":"prc", "domain":"atmos", "interval":"3hr"},
#                   {"var_name":"tos", "domain":"ocean", "interval":"day"},
#                   {"var_name":"orog", "domain":"atmos", "interval":"fx"},
#                   {"var_name":"sftlf", "domain":"atmos", "interval":"fx"},

# DESIRED MODEL LIST
# GCMList=['ACCESS1-0','ACCESS1-3','bcc-csm1-1','bcc-csm1-1-m','BNU-ESM','CanESM2','CCSM4','CESM1-BGC', \
#     'CESM1-CAM5','CMCC-CM','CMCC-CMS','CNRM-CM5','CSIRO-Mk3-6-0','EC-EARTH','FGOALS-g2','FIO-ESM', \
#     'GFDL-CM3','GFDL-ESM2G','GFDL-ESM2M','GISS-E2-H-cc','GISS-E2-H','GISS-E2-R-cc','GISS-E2-R', \
#     'GISS-E2-R','HadGEM2-AO','HadGEM2-CC','HadGEM2-ES','HadGEM2-ES','inmcm4','IPSL-CM5A-LR','IPSL-CM5A-LR', \
#     'IPSL-CM5A-LR','IPSL-CM5A-LR','IPSL-CM5A-MR','IPSL-CM5B-LR','MIROC-ESM','MIROC-ESM-CHEM','MIROC5', \
#     'MPI-ESM-LR','MPI-ESM-MR','MRI-CGCM3','NorESM1-M']

# MODEL LIST ON CHEYENNE
# chyModelList=['BCC/bcc-csm1-1','BCC/bcc-csm1-1-m','BNU/BNU-ESM','CCCma/CanESM2','CNRM-CERFACS/CNRM-CM5','CSIRO-BOM/ACCESS1-0','CSIRO-BOM/ACCESS1-3', \
#               'CSIRO-QCCCE/CSIRO-Mk3-6-0','IPSL/IPSL-CM5A-LR','IPSL/IPSL-CM5A-MR','IPSL/IPSL-CM5B-LR', \
#               'LASG-CESS/FGOALS-g2','MIROC/MIROC5','MIROC/MIROC-ESM-CHEM','MIROC/MIROC-ESM','MOHC/HadGEM2-ES','MRI/MRI-CGCM3', \
#               'NCC/NorESM1-M','NOAA-GFDL/GFDL-CM3','NOAA-GFDL/GFDL-ESM2G']

#####################################################
############## CHANGE THIS INFORMATION ############## 
#####################################################

Institute   = 'BCC'
Model       = 'bcc-csm1-1'  # ACCESS1-0
time_period = 'historical'  # historical, rcp45, rcp85
ensemble    = 'r1i1p1'      # r1i1p1
version     = 'v1'
lat_bnds, lon_bnds = [22, 58], [230, 265]

start_date = "19500101"
end_date   = "20060101"
startDates = pd.to_datetime(['1950-01-01 12:00:00','1960-01-01 06:00:00','1970-01-01 06:00:00','1980-01-01 06:00:00','1990-01-01 06:00:00','2000-01-01 06:00:00'],\
                       format='%Y-%m-%d %H:%M:%S')
endDates   = pd.to_datetime(['1960-01-01 00:00:00','1970-01-01 00:00:00','1980-01-01 00:00:00','1990-01-01 00:00:00','2000-01-01 00:00:00','2006-01-01 00:00:00'],\
                       format='%Y-%m-%d %H:%M:%S')

#####################################################
############## NO NEED TO CHANGE BELOW ############## 
#####################################################

# Check if we've already created a directory for subsetted/processed data
outDir = '/glade/scratch/currierw/'+Model+'/'+time_period+'/' # must have back slash
if os.path.isdir(outDir) == False : # Make output directory if it doesn't exist
    os.makedirs(outDir)
    print("created out directory: "+outDir)
print("Out directory already exists: "+outDir)

# See if data exists on Cheyenne already
# Note: .txt file is important for figuring this out - just searches .txt function
#       find /glade/collections/cmip/cmip5/output1/*/*/*/6hr/atmos/6hrLev/r1i1p1/*/* -type d > .txt
fDirs,varDir,varFiles = cmipFunctions.searchChyColl("/glade/u/home/currierw/cmip_work/CMIP5historical_r1i1p1.txt",Model,'ta') # var not important here, use fDirs
if len(fDirs)==0:
    print('Not on Cheyenne: Need to download all variables')
    chyColl     = False         # Data exist in cheyenne collection already - see list
else:
    print('Data are already on Cheyenne! Yay!')
    chyColl     = True         # Data exist in cheyenne collection already - see list
    inDir       = os.path.dirname(fDirs[0])+'/' # need backward slash at end
print("Cheyenne data are located here: "+inDir)

In [ ]:
# CHECK IF FILES EXIST
cmipFunctions.checkFiles(inDir,'hus',time_period,chyColl)
cmipFunctions.checkFiles(inDir,'ps',time_period,chyColl)
cmipFunctions.checkFiles(inDir,'ta',time_period,chyColl)
cmipFunctions.checkFiles(inDir,'ua',time_period,chyColl)
cmipFunctions.checkFiles(inDir,'va',time_period,chyColl)
# cmipFunctions.convertTos(inDir,outDir,Model,time_period,chyColl)
# cmipFunctions.checkFiles(inDir,'tos',time_period,chyColl)

In [ ]:
####### TEMPERATURE
# Load in Temperature Data First
taFiles=glob.glob(outDir+'ta*.nc')
if len(taFiles) == 0:
    cmipFunctions.loadNonStaggeredVars(inDir,outDir,'ta',Model,time_period,ensemble,lat_bnds,lon_bnds,startDates,endDates,chyColl)

In [ ]:
####### SEA SURFACE TEMEPRATURE

# daily, historical, rcp45, rcp85, sea surface temperature data doesn't exist on cheyenne - download first
tosFiles=glob.glob(outDir+'tos*.nc')
if len(tosFiles) == 0 :
    os.chdir(outDir)
    get_dataset.download(model=Model,var_name="tos",domain="ocean",interval="day", run=ensemble, scenario=time_period, start_time=start_date, end_time=end_date)
# Convert the TOS data from rotated pole to temperature data
tosFilesRegrd=glob.glob(outDir+'tos_6hr*_regrd.nc')
if len(tosFilesRegrd):
    cmipFunctions.convertTos(outDir,inDir,chyColl)
    
# Convert the
tosFilesSub=glob.glob(outDir+'tos_6hr*_regrd_subset.nc')
if len(tosFilesSub) == 0:
    cmipFunctions.processTOS(outDir,outDir,'tos',Model,time_period,ensemble,lat_bnds,lon_bnds,startDates,endDates)

In [ ]:
###### Humidity Data
husFiles=glob.glob(outDir+'hus*.nc')
if len(husFiles) == 0:
    cmipFunctions.loadNonStaggeredVars(inDir,outDir,'hus',Model,time_period,ensemble,lat_bnds,lon_bnds,startDates,endDates,chyColl)

In [ ]:
# ds=xr.open_dataset('/glade/collections/cmip/cmip5/output1/BCC/bcc-csm1-1/historical/6hr/atmos/6hrLev/r1i1p1/v1/ua/ua_6hrLev_bcc-csm1-1_historical_r1i1p1_198101010000-198112311800.nc')


In [ ]:
###### U,V Data
uFiles=glob.glob(outDir+'ua*.nc')
if len(uFiles) == 0:
    cmipFunctions.loadStaggeredVars(inDir,outDir,'ua',Model,time_period,ensemble,lat_bnds,lon_bnds,startDates,endDates,chyColl)
vFiles=glob.glob(outDir+'va*.nc')
if len(vFiles) == 0:
    cmipFunctions.loadStaggeredVars(inDir,outDir,'va',Model,time_period,ensemble,lat_bnds,lon_bnds,startDates,endDates,chyColl)

In [ ]:
###### Surface Pressure
psFiles=glob.glob(outDir+'ps*.nc')
if len(psFiles) == 0:
    cmipFunctions.loadNonStaggeredVars(inDir,outDir,'ps',Model,time_period,ensemble,lat_bnds,lon_bnds,startDates,endDates,chyColl)

In [ ]:
###### Precipitation
# daily, historical, rcp45, rcp85, sea surface temperature data doesn't exist on cheyenne - download first
prcFiles=glob.glob(outDir+'prc*.nc')
if len(prcFiles) == 0 :
    os.chdir(outDir)
    get_dataset.download(model=Model,var_name="prc",domain="atmos",interval="3hr", run=ensemble, scenario=time_period, start_time=start_date, end_time=end_date)

start = ['1960-01-01 06:00:00','1970-01-01 06:00:00','1980-01-01 06:00:00','1990-01-01 06:00:00']
end   = ['1970-01-01 00:00:00','1980-01-01 00:00:00','1990-01-01 00:00:00','2000-01-01 00:00:00']
startDatesPrc = pd.to_datetime(start, format='%Y-%m-%d %H:%M:%S')
endDatesPrc   = pd.to_datetime(end,   format='%Y-%m-%d %H:%M:%S')
# Precipitation is 3D, only time,lat,lon - stored in longer time files in 3 hr data
prcFiles=glob.glob(outDir+'prc*subset.nc')
if len(prcFiles) == 0:
    cmipFunctions.loadNonStaggeredVarsResample(outDir,outDir,'prc',Model,time_period,ensemble,lat_bnds,lon_bnds,startDatesPrc,endDatesPrc,False,'6hr','sum')

In [ ]:
ds=xr.open_dataset('/glade/scratch/currierw/bcc-csm1-1/historical/prc_3hr_bcc-csm1-1_historical_r1i1p1_198001010130-199912312230.nc')
ax = plt.axes(projection=ccrs.PlateCarree())
p = ds['prc'][0,:,:].plot(x='lon', y='lat',transform=ccrs.PlateCarree(),subplot_kws={'projection': ccrs.PlateCarree()})
ax.coastlines();ax.gridlines();ax.add_geometries(states_gdf.geometry, crs = ccrs.PlateCarree(),facecolor='none', edgecolor='black')

In [ ]:
dsT    = xr.open_mfdataset(outDir+'ta*subset.nc',combine='by_coords')
dsTOS  = xr.open_mfdataset(outDir+'tos*subset.nc',combine='by_coords')
dsHus  = xr.open_mfdataset(outDir+'hus*subset.nc',combine='by_coords')
dsU    = xr.open_mfdataset(outDir+'ua*subset.nc',combine='by_coords')
dsV    = xr.open_mfdataset(outDir+'va*subset.nc',combine='by_coords')
dsPrc  = xr.open_mfdataset(outDir+'prc*subset.nc',combine='by_coords')
dsPrcO = xr.open_mfdataset(outDir+'prc_3hr*.nc',combine='by_coords')
dsPs   = xr.open_mfdataset(outDir+'ps*subset.nc',combine='by_coords')

In [ ]:
# Calculate the model level heights [m] or Pressure Levels
try:
    z_t=dsT.lev+(dsT.b*dsT.orog)
    print('\nCalculating Model Level Heights')
except:
    P = dsT['a']*dsT['p0'] + dsT['b']*dsT['ps']
    pressure  = True
    elevation = False
else:
    elevation = True
    pressure  = False

In [ ]:
# Calculate water vapor mixing ratio (w/Qv) [kg/kg] from specific humdiity
print('\n Converting from Specific Humidity to Water Vapor Mixing Ratio, Qv = hus/1-hus')
dsHus['Qv'] = dsHus['hus']/(1-dsHus['hus'])

In [ ]:
# Creating New Dataset
print("\nCreating new dataset\n")

# Create New Datasets that's just the datasets with dimensions and coordinates

######## Terrain Height
if elevation:
    dsOrog = xr.Dataset({"HGT":(("lat","lon"),tads_sub.orog)},
                     coords={"lat":tads_sub.lat,"lon":tads_sub.lon})
    dsOrog['HGT'].attrs['standard_name'] = 'Terrain Height'
    dsOrog['HGT'].attrs['long_name']     = 'Terrain Height'
    dsOrog['HGT'].attrs['units']         = 'm'
    dsOrog['HGT'].attrs['Processing Note'] = 'Provided from temeprature dataset - orogoraphy variable'
    ######## 3D Model Level Heights [m]
    ds_Z    = xr.Dataset({"Z":(("lev","lat","lon"),z_t)},
                 coords={"lev":tads_sub.lev,"lat":tads_sub.lat,"lon":tads_sub.lon})
    ds_Z['Z'].attrs['standard_name'] = '3D model level heights'
    ds_Z['Z'].attrs['long_name'] = '3D Model Level Heights'
    ds_Z['Z'].attrs['units'] = 'm' 
    ds_Z['Z'].attrs['Processing Note'] = 'Calculated using temperature lev coordinates, b values, and orogoraphy varaibles. Z = lev + b * orog' 

if pressure:
    ds_P = xr.Dataset({"P":(("time","lev","lat","lon"),P)},
                 coords={"time":dsT.time,"lev":P.lev,"lat":P.lat,"lon":P.lon})
    ds_P['P'].attrs['standard_name'] = 'pressure' 
    ds_P['P'].attrs['long_name'] = 'pressure' 
    ds_P['P'].attrs['comment'] = 'calculated from subsetted 6 hr temperature data file: a*p0+b*Ps' 
    ds_P['P'].attrs['units'] = 'Pa' 

    
######## Surface Air Pressure [Pa]
ds_Ps   = xr.Dataset({"Ps":(("time","lat","lon"),dsPs.ps)},
                 coords={"time":dsPs.time,"lat":dsPs.lat,"lon":dsPs.lon})
ds_Ps['Ps'].attrs = dsPs.ps.attrs

######## Sea Surface Temeperature [K]
ds_SST  = xr.Dataset({"SST":(("time","lat","lon"),dsTOS.tos)},
                 coords={"time":dsTOS.time,"lat":dsT.lat,"lon":dsT.lon})
ds_SST['SST'].attrs = dsTOS['tos'].attrs
ds_SST['SST'].attrs['Processing Note'] = 'Daily data forward filled to 6H data' 

######## Precipitation [kg m-2 s-1]
ds_prec = xr.Dataset({"prec":(("time","lat","lon"),dsPrc.prc)},
                 coords={"time":dsPrc.time,"lat":dsPrc.lat,"lon":dsPrc.lon})
ds_prec['prec'].attrs['standard_name'] =dsPrcO['prc'].attrs['standard_name']
ds_prec['prec'].attrs['long_name'] = dsPrcO['prc'].attrs['long_name']
ds_prec['prec'].attrs['comment'] = 'at surface. This is a 6-hour mean convective preciptiation'
ds_prec['prec'].attrs['units'] = dsPrcO['prc'].attrs['units']
ds_prec['prec'].attrs['cell_methods'] = dsPrcO['prc'].attrs['cell_methods']
ds_prec['prec'].attrs['cell_measures'] = dsPrcO['prc'].attrs['cell_measures']
ds_prec['prec'].attrs['associated_files'] = dsPrcO['prc'].attrs['associated_files']
ds_prec['prec'].attrs['Processing Note'] = 'Resample from 3H data to 6H data using the sum between time-steps'

######## Air Temperature [K[]]
ds_T = xr.Dataset({"T":(("time","lev","lat","lon"),dsT.ta)},
                  coords={"time":dsT.time,"lev":dsT.lev,"lat":dsT.lat,"lon":dsT.lon})
ds_T['T'].attrs = dsT.ta.attrs

######## Water Vapor Mixing Ratio [kg/kg]
ds_Qv = xr.Dataset({"Qv":(("time","lev","lat","lon"),dsHus.Qv)},
                   coords={"time":dsHus.time,"lev":dsHus.lev,"lat":dsHus.lat,"lon":dsHus.lon})
ds_Qv['Qv'].attrs['standard_name'] = 'Water Vapor Mixing Ratio'
ds_Qv['Qv'].attrs['long_name'] = 'Water Vapor Mixing Ratio'
ds_Qv['Qv'].attrs['units'] = 'kg/kg'
ds_Qv['Qv'].attrs['cell measures'] = dsHus['hus'].attrs['cell_measures']
ds_Qv['Qv'].attrs['associated_files'] = dsHus['hus'].attrs['associated_files']
ds_Qv['Qv'].attrs['Processing Note'] = 'Calculated from specific humidity (q) data Qv = q/1-q'

######## North South Wind Speeds [m s-1]
ds_v = xr.Dataset({"V":(("time","lev","lat","lon"),dsV.va)},
                  coords={"time":dsV.time,"lev":dsT.lev,"lat":dsV.lat,"lon":dsV.lon})
ds_v['V'].attrs = dsV.va.attrs
ds_v['V'].attrs['Processing Note'] = 'Interpolated/Regrided from staggered north-south grid - one additional row offset by 1/2 a grid cell in latitude to the temperature grid'

######## East Wind Wind Speeds [m s-1]
ds_u = xr.Dataset({"U":(("time","lev","lat","lon"),dsU.ua)},
                  coords={"time":dsU.time,"lev":dsT.lev,"lat":dsU.lat,"lon":dsU.lon})
ds_u['U'].attrs = dsU.ua.attrs
ds_u['U'].attrs['Processing Note'] = 'Interpolated/Regrided from staggered east-west grid - one additional column offset by 1/2 a grid cell in longitude to the temperature grid'
                     
# Make a new dataset
if elevation:
    ds = xr.merge([ds_orog, ds_Z, ds_Ps, ds_SST, ds_prec, ds_T, ds_Qv, ds_v, ds_u])
elif pressure:
    ds = xr.merge([ds_P, ds_Ps, ds_SST, ds_prec, ds_T, ds_Qv, ds_v, ds_u])
    
print("Created new dataset")
now = datetime.now()
ds.attrs['Condensed/Merged File Created'] = now.strftime("%m/%d/%Y, %H:%M:%S")
ds.attrs = dsT.attrs

In [ ]:
# Temperature
ax = plt.axes(projection=ccrs.PlateCarree())
p = ds['SST'][0,:,:].plot(x='lon', y='lat',transform=ccrs.PlateCarree(),subplot_kws={'projection': ccrs.PlateCarree()})
ax.coastlines();ax.gridlines();ax.add_geometries(states_gdf.geometry, crs = ccrs.PlateCarree(),facecolor='none', edgecolor='black')

In [ ]:
# Temperature
ax = plt.axes(projection=ccrs.PlateCarree())
p = ds['T'][0,0,:,:].plot(x='lon', y='lat',transform=ccrs.PlateCarree(),subplot_kws={'projection': ccrs.PlateCarree()})
ax.coastlines();ax.gridlines();ax.add_geometries(states_gdf.geometry, crs = ccrs.PlateCarree(),facecolor='none', edgecolor='black')

In [ ]:
# Write out files
allFiles=glob.glob(outDir+Model+'_6hrLev_'+ time_period +'_'+ ensemble +'*' +'_subset.nc')
if len(allFiles) == 0:
    for startDateLs, endDateLs in zip(startDates, endDates):
        ds_sub_Time  = ds.sel(time=slice(startDateLs.strftime('%Y-%m-%dT%H:%M:%S'),endDateLs.strftime('%Y-%m-%dT%H:%M:%S')))  # Time slice
        ds_sub_Time.to_netcdf(outDir+Model+'_6hrLev_'+ time_period +'_'+ ensemble +'_' \
                                + startDateLs.strftime('%Y%m%d')+'-'+endDateLs.strftime('%Y%m%d') \
                                +'_subset.nc')

In [ ]:
ds=xr.open_dataset(allFiles[1])
ds